<h2>Acceptor</h2>

The task is to realize acceptor class using Python. It has to have `accept(word)` method, that returns `True`, if acceptor accepts `word` else `False`.

Acceptor is $(\sum, S, s_0 \in S, F, \sigma:S \times \sum \to S)$, where:

- $\sum$ - alphabet;

- $S$ - set of available instances;

- $s_0 \in S$ - default instance;

- $F$ - final instances;

- $\sigma$ - map with $S \times \sum$ and $S$;

The example to test acceptor class is to find out, is len of string with symbols `('a', 'b', 'c')` is even.

Program realization:
- `accept()` was realized int two ways: recursive and iterative;
- `delete_unreachable_instances()` was realized after `Acceptor` class;

In [1]:
from collections.abc import Iterable

class Acceptor(object):
    '''
    Program realization of simple acceptor. 
    Since all instances are elements of set, they are hashable.
    
    Properties defined here:
        - sigma:set;
        - instances:set;
        - s0:object;
        - final_instances:set;
        - omega:dict
        
    You can get property like:
        instances:set = acceptor.instances
    But can not set property like:
        acceptor.sigma = {0, 1, 2}
        
    Methods defined here:
        accept(word) -> bool: returns is last instance for word is final;
    '''
    def __init__(self, sigma:set=None,
                       instances:set=None,
                       s0:object=None,
                       final_instances:set=None,
                       omega:dict=None):
        '''
        sigma - set of tokens (it is alphabet). 
        instances - set of instances.
        s0 - default instance. Should be in instances.
        final_instances - set of final instances. Should be subset of instances.
        omega - map for processing. Keys should be like (s, e), where:
            - s is in instances
            - e is in sigma
        Values of omega should be in instances.

        Also, s0 should be at list once in keys.
        '''
        
        # validate is sigma set 
        if not isinstance(sigma, set):
            raise TypeError('sigma argument is not of type set. Please, use set().')
            
        # validate is instances set
        if not isinstance(instances, set):
            raise TypeError('instances argument is not of type set. Please, use set().')
            
        # check is s0 in instances
        if s0 not in instances:
            raise ValueError('Default instance is not in instances set.')
            
        # check is final instances is set
        if not isinstance(final_instances, set):
            raise TypeError('final_instances is not of type set. Please, use set().')
            
        # check, is final instances subset of instances
        for instance in final_instances:
            if instance not in instances:
                raise ValueError(f'{instance} from final_instances is not in instances.')
            
        # check, is omega dict
        if not isinstance(omega, dict):
            raise TypeError('omega argument is not of type dict.')
            
        # if s0 is not in omega, acceptor can not work
        s0_in_omega:bool = False
            
        # keys of omega should be like (s, e)
        # values of omega should be in instances
        for key, value in omega.items():
            
            # key should be of type tuple
            if not isinstance(key, tuple):
                raise TypeError(f'{key} (type {type(key)}) is not of type tuple.')
                
            # key should be like (s, e)
            if len(key) != 2:
                raise TypeError(f'Keys in omega should be tuple with size 2, not {len(key)} ({key}).')
                
            # s should be in instances
            # e should be in sigma
            s, e = key
            
            if s not in instances:
                raise ValueError(f's in {key} is not in instances.')
                
            if e not in sigma:
                raise ValueError(f'e in {key} is not in sigma.')
                
            # value should be in instances
            if value not in instances:
                raise ValueError(f'{value} is not in sigma.')
                
            # search s0
            s0_in_omega:bool = (s == s0) or s0_in_omega

        # s0 is not in omega
        if not s0_in_omega:
            raise ValueError('s0 is not available in omega.')
                
        # create new instance of class
        self.__sigma:set = sigma
        self.__instances:set = instances
        self.__s0:object = s0
        self.__final_instances:set = final_instances
        self.__omega:dict = omega
            
            
    def accept(self, word:Iterable, recursive:bool=False) -> bool:
        '''
        Main method of acceptor. 
        Returns True, if last instance for word is final,
        else False.

        Uses two realizations, recursive and iterative.
        '''

        try:
            # convert to list for usability
            word:list = list(word)
        # word is not iterable
        except:
            raise TypeError('Argument in accept() is not iterable.')

        # recursive realization
        if recursive:
            def get_instance(cur_inst:object, word:list) -> object:
                '''
                Inner function, that gets last 
                '''
                # finish recursion
                if len(word) == 0:
                    return cur_inst
                else:
                    # get next instance
                    cur_inst:object = self.__omega[(cur_inst, word[0])]

                    # go to next token
                    return get_instance(cur_inst, word[1:])
        # iterative realization
        else:
            def get_instance(cur_inst:object, word:list) -> object:

                for token in word:
                    cur_inst:object = self.__omega[(cur_inst, token)]

                return cur_inst

        # get last instance
        last_inst:object = get_instance(self.__s0, word)

        return last_inst in self.__final_instances
       
    @property
    def sigma(self) -> set:
        '''
        Property to get sigma. Has not setter.
        '''
        return self.__sigma
    
    @property
    def instances(self) -> set:
        '''
        Property to get instances. Has not setter.
        '''
        return self.__instances
    
    @property
    def s0(self) -> object:
        '''
        Property to get s0. Has not setter.
        '''
        return self.__s0
    
    @property
    def final_instances(self) -> set:
        '''
        Property to get final_instances. Has not setter.
        '''
        return self.__final_instances
    
    @property
    def omega(self) -> set:
        '''
        Property to get omega. Has not setter
        '''
        return self.__omega

<h4>Example</h4>

Find out, is len of string with symbols `('a', 'b', 'c')` is even.

In [2]:
# create automaton
sigma:set = set(['a', 'b', 'c'])
instances:set = set(['even', 'odd'])
s0:str = 'odd'
final_instances:set = set(['even'])
omega:dict = {('even', 'a'):'odd',
              ('even', 'b'):'odd',
              ('even', 'c'):'odd',
              ('odd', 'a'):'even',
              ('odd', 'c'):'even',
              ('odd', 'b'):'even'}

automaton:Acceptor = Acceptor(sigma, 
                              instances, 
                              s0, 
                              final_instances, 
                              omega)
    
# test data
test_data:list = ['aaaabbcbab', 
                  'aaa', 
                  'abacbab', 
                  'abcbabcb', 
                  'bcbabc', 
                  'bacbabcb', 
                  'ccbabcbab']
    
# get acceptions
tuple(automaton.accept(word) for word in test_data)

(False, True, True, False, False, False, True)

<h2>Delete unreachable instances</h2>
The next task is to realize function, that passes acceptor as an argument returns new acceptor, instances of which are all reachable.

The algorithm:
- mark every instance as `False`;
- define `s` as `s0`;
- iterate over all symbols ($a$) in sigma:
    
    if $\sigma(s, a) = False \Rightarrow \sigma(s, a) = True; \text{s = } \sigma \text{(s, a)}$ 
    
Recursion in this algorithm stops, when all reachable instances are marked as `True`.

Program realization:

In [3]:
def delete_unreachable_instances(acceptor:Acceptor) -> Acceptor:
    '''
    Deletes all unreachable instances from acceptor.instances
    and returns new acceptor withput those instances.
    '''
    sigma:set = acceptor.sigma
    instances:set = acceptor.instances
    s0:object = acceptor.s0
    omega:dict = acceptor.omega
        
    marking:dict = {instance:False for instance in instances}
        
    def mark(s:object) -> None: 
        '''
        Inner function, that marks s as visited,
        iterates over all tokens and if new
        instance is not visited, calls mark(new_instance).
        '''
        # mark instance as visited
        marking[s]:bool = True
            
        # iterate over all tokens
        for token in sigma:
            if not marking[omega[(s, token)]]:
                mark(omega[(s, token)])
                
    # run for default instance - s0
    mark(s0)
    
    # build new instances
    instances:set = {key for key in marking.keys() if marking[key]}
    
    # create new instance of Acceptor
    return Acceptor(sigma,
                    instances,
                    s0,
                    acceptor.final_instances,
                    omega)

In [4]:
# create automaton
sigma:set = set(['a', 'b', 'c'])
instances:set = set(['even', 'odd', 1])
s0:str = 'odd'
final_instances:set = set(['even'])
omega:dict = {('even', 'a'):'odd',
              ('even', 'b'):'odd',
              ('even', 'c'):'odd',
              ('odd', 'a'):'even',
              ('odd', 'c'):'even',
              ('odd', 'b'):'even'}
    
a:Acceptor = Acceptor(sigma, 
                              instances, 
                              s0, 
                              final_instances, 
                              omega)
    
a = delete_unreachable_instances(a)
a.instances

{'even', 'odd'}

<h4>Example</h4>

Find out, is notation of number with exponent is correct. For simplification, use only `0` and `1` digits.

Legal notation is like `'10101...10101.010...110001e10101001...'` without `+` or `-`.
Rules:
- `0` can not be first token;
- `.` can not be first token;
- `e` can not be before `.`
- `.` must be before `e`
- after `.` must be at list one token;
- after `e` must be at list one token;
- after `e` can not be `.`;

In [5]:
# build auromaton
sigma:set = set(['0', '1', '.', 'e'])
instances:set = set(['begin', 
                     'read_whole_part',
                     'point_readed', 
                     'read_after_point', 
                     'e_readed',
                     'read_after_e', 
                     'error',
                     'unreachable_inst_1',
                     'unreachable_inst_2'])
s0:str = 'begin'
final_instances = set(['begin', 
                       'read_whole_part', 
                       'read_after_point', 
                       'read_after_e'])

omega:dict = {
    ('begin', '0'):'error',
    ('begin', '1'):'read_whole_part',
    ('begin', 'e'):'error',
    ('begin', '.'):'error',
    ('read_whole_part', '0'):'read_whole_part',
    ('read_whole_part', '1'):'read_whole_part',
    ('read_whole_part', 'e'):'error',
    ('read_whole_part', '.'):'point_readed',
    ('point_readed', '0'):'read_after_point',
    ('point_readed', '1'):'read_after_point',
    ('point_readed', 'e'):'error',
    ('point_readed', '.'):'error',
    ('read_after_point', '1'):'read_after_point',
    ('read_after_point', '0'):'read_after_point',
    ('read_after_point', 'e'):'e_readed',
    ('read_after_point', '.'):'error',
    ('e_readed', '0'):'read_after_e',
    ('e_readed', '1'):'read_after_e',
    ('e_readed', '.'):'error',
    ('e_readed', 'e'):'error',
    ('read_after_e', '0'):'read_after_e',
    ('read_after_e', '1'):'read_after_e',
    ('read_after_e', 'e'):'error',
    ('read_after_e', '.'):'error',
    ('error', '0'):'error',
    ('error', '1'):'error',
    ('error', '.'):'error',
    ('error', 'e'):'error',
}
    
acceptor:Acceptor = Acceptor(sigma, 
                             instances, 
                             s0, 
                             final_instances, 
                             omega)
    
# delete unreachable instances
acceptor:Acceptor = delete_unreachable_instances(acceptor)

acceptor.instances

{'begin',
 'e_readed',
 'error',
 'point_readed',
 'read_after_e',
 'read_after_point',
 'read_whole_part'}

In [6]:
# test acceptor
test_data:list = ['1111.1010110e101010',
                  '01111e1111',
                  '1010101',
                  '1010101.0e10101',
                  '.101010',
                  '10101e10101.101010']
    
for word in test_data:
    print(f'{word} is {"correct" if acceptor.accept(word) else "incorrect"}')

1111.1010110e101010 is correct
01111e1111 is incorrect
1010101 is correct
1010101.0e10101 is correct
.101010 is incorrect
10101e10101.101010 is incorrect
